<h2>Assess the performance of the model for the MAPE and MAE metrics on the training, validation and test sets </h2>

In [ ]:
import numpy as np
import keras
from keras.models import load_model
from keras.preprocessing import image
import os
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#list of models where the performance has to be assessed
models = ["/content/drive/My Drive/dataset_AA2/models/model_1_Xception_20epo_1200data.keras",
          "/content/drive/My Drive/dataset_AA2/models/model_1_Xception_50epo_1500data.keras",
          "/content/drive/My Drive/dataset_AA2/models/model_1_Xception_150epo_1500data.keras",
          "/content/drive/My Drive/dataset_AA2/models/model_1_Xception_150epo_1500data_LR_001_Adagrad.keras",
          "/content/drive/My Drive/dataset_AA2/models/model_1_Xception_150epo_1500data_LR_001_Amsgrad.keras",
          "/content/drive/My Drive/dataset_AA2/models/model_1_Xception_150epo_1500data_LR_001.keras",
          "/content/drive/My Drive/dataset_AA2/models/model_1_Xception_150epo_1500data_LR_01.keras"
          ]

In [5]:
#measure the mean percentage difference between the true value and the predicted value
def mean_absolute_percentage_error(y_true, y_pred):

    #filter rows with zero
    non_zero_mask = y_true != 0
    y_true = y_true[non_zero_mask]
    y_pred = np.array(y_pred.reshape(-1))
    y_pred = y_pred[non_zero_mask]

    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [6]:
#measure the mean absolute error difference between the true value and the predicted value
def mean_absolute_error(y_true, y_pred):

    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs(y_true - y_pred))

In [7]:
#create array with all the images
def load_images_from_folder(folder, target_size):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = image.load_img(img_path, target_size=target_size)
        img_array = image.img_to_array(img)
        img_array /= 255.  # Scale pixel values to [0, 1]
        img_array = img_array*2 -1
        if img_array is not None:
            images.append(img_array)
    return np.array(images)

In [8]:
folder_path_test = '/content/drive/My Drive/dataset_AA2/images-processed/test/'
folder_path_train = '/content/drive/My Drive/dataset_AA2/images-processed/train/'
folder_path_validation = '/content/drive/My Drive/dataset_AA2/images-processed/validation/'

folder_paths = [folder_path_train,folder_path_validation, folder_path_test] #folder_path_train,

target_size = (299, 299)  # Set the target size as required by your model

In [9]:
dataset = pd.read_csv('/content/drive/My Drive/dataset_AA2/metadata_v2.csv')

In [10]:
#create array with all the images
def load_true_prices_from_folder(folder):
    prices = []
    for filename in os.listdir(folder):
        price = dataset.loc[dataset["Image Name"]==filename,"Price"].iloc[0]
        if price is not None:
            prices.append(price)
    return np.array(prices)

In [11]:
#assess the performance for all the models on every subdataset

array_MAPE_MAE = np.zeros((len(models),6))

train_val_test_counter = 0

for path in folder_paths:
  images = load_images_from_folder(path, target_size)
  true_values = load_true_prices_from_folder(path)

  model_counter = 0

  for model_name in models:
    model = load_model(model_name)
    predictions = model.predict(images)

    array_MAPE_MAE[model_counter, train_val_test_counter] = mean_absolute_percentage_error(true_values, predictions)
    array_MAPE_MAE[model_counter, train_val_test_counter+3] = mean_absolute_error(true_values, predictions)

    print(array_MAPE_MAE[model_counter, train_val_test_counter])
    print(array_MAPE_MAE[model_counter, train_val_test_counter+3])
    print()
    model_counter +=1

  print(array_MAPE_MAE[:, train_val_test_counter])
  print(array_MAPE_MAE[:, train_val_test_counter+3])
  train_val_test_counter += 1

47/47 [==============================] - 20s 276ms/step
93.2538379420829
1638.7834837300938

47/47 [==============================] - 10s 209ms/step
70.37297582933385
1210.604100494185

47/47 [==============================] - 10s 209ms/step
86.27767849610606
851.8906461654294

47/47 [==============================] - 11s 207ms/step
49.087123251420074
967.0105084404986

47/47 [==============================] - 11s 212ms/step
49.09756916448536
967.0016849480862

47/47 [==============================] - 11s 210ms/step
80.48423394977033
856.6302618798795

47/47 [==============================] - 11s 213ms/step
49.85389473520721
955.2005632960178

[93.25383794 70.37297583 86.2776785  49.08712325 49.09756916 80.48423395
 49.85389474]
[1638.78348373 1210.60410049  851.89064617  967.01050844  967.00168495
  856.63026188  955.2005633 ]
4/4 [==============================] - 2s 226ms/step
91.49998965752268
1627.990291164548

4/4 [==============================] - 1s 226ms/step
87.82361871405988

In [12]:
print(array_MAPE_MAE)

[[  93.25383794   91.49998966   91.92885429 1638.78348373 1627.99029116
  1492.45060736]
 [  70.37297583   87.82361871   56.61375901 1210.60410049 1399.68426768
  1014.27389865]
 [  86.2776785    96.20336056  101.04157139  851.89064617 1160.49089153
   982.77399258]
 [  49.08712325   82.88336326  116.15152937  967.01050844 1238.92460872
  1133.2570504 ]
 [  49.09756916   82.90692836  116.1648843   967.00168495 1238.93249177
  1133.29246087]
 [  80.48423395  102.27028184  112.67621638  856.63026188 1161.9028189
  1040.31710692]
 [  49.85389474   80.60328171  111.19494588  955.2005633  1226.25006324
  1102.67743632]]
